In [1]:
import pandas as pd
import re
import tqdm
import pickle

In [2]:
#2019년, 2020년 지하철 승하차 데이터 로드
sub_19 = pd.read_csv('../data/서울교통공사_1_8호선일별역별시간대별승하차인원_202001_202008.csv', encoding='cp949')
sub_20 = pd.read_excel('../data/서울교통공사 2019년 일별 역별 시간대별 승하차인원(1_8호선).xlsx')

## 요일 칼럼 생성

In [3]:
def make_day(df):
    # 19년 데이터의 경우 컬럼이 제대로 불러와지지 않는 에러 발생
    if 'Unnamed: 0' in df.columns:
        df.columns = df.loc[0]
        df.drop(0, axis=0, inplace=True)
    else:
        pass
    
    df['날짜'] = pd.to_datetime(df['날짜'])
    #datetime에서 요일 불러오기
    df['요일'] = df['날짜'].dt.day_name()
    
    days = list(df['요일'])
    kor_day = []
    for day in days:
        if day == 'Friday':
            kor_day.append('금요일')
        elif day == 'Monday':
            kor_day.append('월요일')
        elif day == 'Saturday':
            kor_day.append('토요일')
        elif day == 'Sunday':
            kor_day.append('일요일')
        elif day == 'Thursday':
            kor_day.append('목요일')
        elif day == 'Tuesday':
                kor_day.append('화요일')
        elif day == 'Wednesday':
            kor_day.append('수요일')

    df['요일'] = kor_day
    
    return df

In [4]:
sub_19 =make_day(sub_19)
sub_20 =make_day(sub_20)

In [31]:
subway = pd.concat([sub_19, sub_20], axis=0)
subway.set_index('날짜', inplace=True)
#날짜순으로 데이터 정렬
subway.sort_index(inplace=True)
subway

,호선,역번호,역명,구분,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,...,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후,요일,합 계
날짜,,,,,,,,,,,,,,,,,,,,,
2019-01-01,6호선,2647,화랑대(서울여대입구),하차,22,119,69,130,159,206,...,292,377,277,272,277,366,234,34,화요일,4249
2019-01-01,6호선,2629,삼각지,하차,46,89,64,80,105,138,...,154,128,152,157,116,152,80,14,화요일,2411
2019-01-01,6호선,2629,삼각지,승차,48,68,51,80,110,98,...,376,269,150,125,100,74,50,0,화요일,2981
2019-01-01,6호선,2628,효창공원앞,하차,24,76,64,103,129,110,...,214,213,179,149,163,145,80,12,화요일,2533
2019-01-01,6호선,2628,효창공원앞,승차,52,104,94,109,143,175,...,193,188,121,102,100,77,25,0,화요일,2797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,4호선,409,당고개,승차,294,720,2032,1780,944,504,...,449,432,307,200,173,110,27,2,월요일,NaN
2020-08-31,4호선,409,당고개,하차,0,245,305,451,319,369,...,750,1512,1321,810,556,397,215,13,월요일,NaN
2020-08-31,4호선,410,상계,승차,471,1369,3415,2747,1212,744,...,533,635,371,278,224,180,56,1,월요일,NaN


## 탐색의 편의성을 위해 시간 칼럼 수정

In [32]:
time_col = list(subway.columns[4:-2])

In [33]:
time_col

['06시 이전',
 '06 ~ 07',
 '07 ~ 08',
 '08 ~ 09',
 '09 ~ 10',
 '10 ~ 11',
 '11 ~ 12',
 '12 ~ 13',
 '13 ~ 14',
 '14 ~ 15',
 '15 ~ 16',
 '16 ~ 17',
 '17 ~ 18',
 '18 ~ 19',
 '19 ~ 20',
 '20 ~ 21',
 '21 ~ 22',
 '22 ~ 23',
 '23 ~ 24',
 '24시 이후']

In [34]:
new_time = []
for timeline in time_col:
    if '0' in timeline[0]:
        timeline = re.sub('0', '', timeline)
    else:
        pass
    timeline = re.sub(' ~ ','-', timeline)
    new_time.append(timeline)

In [35]:
new_time[0], new_time[4], new_time[-1] = '5-6', '9-10', '24-01'

In [36]:
col_all = list(subway.columns)

In [37]:
col_all[4:-2] = new_time #지하철 데이터의 시간 칼럼

In [38]:
subway.columns = col_all

## 공휴일 처리 문제
- 주중이면 0, 주말 & 공휴일이면 1

In [39]:
subway['공휴일'] = 0

In [40]:
temp_holiday = subway[['요일', '공휴일']]

In [41]:
temp_holiday.reset_index(inplace=True)

In [42]:
temp_holiday

,날짜,요일,공휴일
0,2019-01-01,화요일,0
1,2019-01-01,화요일,0
2,2019-01-01,화요일,0
3,2019-01-01,화요일,0
4,2019-01-01,화요일,0
...,...,...,...
335379,2020-08-31,월요일,0
335380,2020-08-31,월요일,0
335381,2020-08-31,월요일,0
335382,2020-08-31,월요일,0


## 토요일과 일요일 공휴일 처리

In [43]:
sun_index = temp_holiday[temp_holiday['요일'] == '일요일']['공휴일'].index
temp_holiday['공휴일'][sun_index] = 1

sat_index = temp_holiday[temp_holiday['요일'] == '토요일'].index
temp_holiday['공휴일'][sat_index] = 1

C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\pandas\core\series.py:1092: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_labels(key, value)
C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_g

In [44]:
subway.reset_index(inplace=True)
subway['공휴일'] = temp_holiday['공휴일']

In [45]:
subway.set_index('날짜', inplace=True)

# 공공데이터 공휴일 정보 ( api key필요)
- 특일정보 (https://www.data.go.kr/data/15012690/openapi.do)

In [46]:
import requests
from bs4 import BeautifulSoup

def get_request_query(url, operation, params, serviceKey):
    import urllib.parse as urlparse
    params = urlparse.urlencode(params)
    request_query = url + '/' + operation + '?' + params + '&' + 'serviceKey' + '=' + serviceKey
    return request_query

URL = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
OPERATION = 'getHoliDeInfo' # 국경일 & 공휴일 정보 조회

SERVICEKEY = 'api키 필요'
solYear  = '2020'  #2020년 공휴일 정보 반환
PARAMS = {'solYear':solYear}

request_query = get_request_query(URL, OPERATION, PARAMS, SERVICEKEY)
response = requests.get(url=request_query)

soup = BeautifulSoup(response.content, 'lxml')
data = soup.find_all('item')
holiday_list = []

for item in data:
    datename = item.find('datename')
    locdate = item.find('locdate')
    holiday_list.append([datename.get_text(), locdate.get_text()])
    
holiday = pd.DataFrame(holiday_list, columns = ['공휴일', '날짜'])
holiday_date = list(holiday['날짜'])

#datetime 형식에 맞게 변환
date_modi = []
for holi in holiday_date:
    date_modi.append(holi[:4] + '-' + holi[4:6] + '-' + holi[6:])

holiday['날짜'] = date_modi

for date in holiday['날짜']: 
    subway.loc[date]['공휴일'] = 1

C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## 지하철 예외처리
- 동대문역사문화공원, 용마산, 낙성대의 경우 3월에 역명이 변경

In [47]:
subway.reset_index(inplace=True)

In [48]:
# 동대문역사문화공원
ddp_index = subway[subway['역명'] == '동대문역사문화공원(DDP)'].index
subway.loc[ddp_index, '역명'] = '동대문역사문화공원'

#용마산
yong_index = subway[subway['역명'] == '용마산(용마폭포공원)'].index
subway.loc[yong_index, '역명'] = '용마산'

#낙성대
nack_index = subway[subway['역명'] == '낙성대(강감찬)'].index
subway.loc[nack_index, '역명'] = '낙성대'

In [49]:
#하차 데이터 생성
down = subway[subway['구분'] == '하차']

#승차 데이터 생성
up = subway[subway['구분'] == '승차']

In [50]:
# 2020년 데이터만
down = down.loc[200805:]

#인덱싱을 위해 날짜, 역명 칼럼만 추출, 중복 제거
down = down[['날짜', '역명']]
down = down.drop_duplicates()

#시간 칼럼 생성
for time in time_col:
    down[time] = 0
    
down = down.groupby(['역명', '날짜']).sum()

In [51]:
subway.rename(columns={'날짜':'date'}, inplace=True)

In [52]:
subway.set_index('date', inplace=True)

결과 값 pkl형태로 저장

In [53]:
# subway.to_pickle('data/지하철데이터.pkl')